In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 204.2MB 67kB/s 
     |████████████████████████████████| 204kB 52.4MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=7334c08e1113b74b8866b20be2cc18264f617a3d7a9d8e296e7ea68771fa0476
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 15 not upgraded.
Need to get 36.5 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id='1f_9EbnywCj35EBUA32sueigxBjBJwALr'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('War and Peace by Leo Tolstoy (ru).txt')

In [ ]:
import pandas as pd

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [ ]:
conf = SparkConf().set("spark.ui.port", "4050")

sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [ ]:
spark

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2020-12-23 14:37:49--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.213.190.117, 54.160.18.204, 52.202.213.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.213.190.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  18.4MB/s    in 0.7s    

2020-12-23 14:37:50 (18.4 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
Traceback (most recent call last):
  File "<string>", line 1, in <module>
IndexError: list index out of range


Подсчитай кол-во слов в документе "War and Peace by Leo Tolstoy (ru).txt"

In [ ]:
from pyspark.sql import SparkSession

ss = SparkSession(sc)

In [ ]:
import re
f = open("War and Peace by Leo Tolstoy (ru).txt", "rb")
content = re.sub(r'[^\w\s]+|[\d]+', r'',f.read().decode("cp1251").encode("utf-8").decode("utf-8")).strip().replace("\xa0","").replace("\t\t","")
content = re.sub(r"\b[A-Z]{1,}\b", "", content)
f.close()
f = open("War and Peace.txt", "w")
f.write(content)
f.close()

In [ ]:
book = sc.textFile("War and Peace.txt")

In [ ]:
words = book.flatMap(lambda line: line.split(" ")).filter(lambda char: char != '')
wordCounts = words.map(lambda word: (word, 1)).reduceByKey(lambda a,b:a +b)


In [ ]:
wordCounts.values().sum()

470745

In [ ]:
id='13yfAoONwq4rS5XrTv3IrcqcFcdgfvK9V'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('mnist-digits-train.txt')

id='1VE_9x0LQvOJpHXbXp_RMPl3Q4wRUuOok'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('mnist-digits-test.txt')

Необходимо обучить модель используя Spark MLlib (модель на ваш выбор, например Decision Tree) и получить accuracy.
Подробнее тут: https://spark.apache.org/docs/latest/ml-classification-regression.html

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Load the data stored in LIBSVM format as a DataFrame.
data = spark.read.format("libsvm").option("numFeatures","780").load("mnist-digits-train.txt")
data_test = spark.read.format("libsvm").option("numFeatures","780").load("mnist-digits-test.txt")
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", handleInvalid="skip").fit(data)
dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")
pipeline_dt = Pipeline(stages=[labelIndexer, featureIndexer, dt])
model_dt = pipeline_dt.fit(data)

predictions_dt = model_dt.transform(data_test)
evaluator_dt = MulticlassClassificationEvaluator(
    labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy_dt = evaluator_dt.evaluate(predictions_dt)
print("Accuracy DTC = %f " % accuracy_dt)


Accuracy DTC = 0.683930 


In [ ]:
id='1kUIrskM0zNH8u71G9M1BkHjRQYxvgAvh'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('data.zip')
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: data/regions.csv        
  inflating: data/departments.csv    
  inflating: data/jobs.csv           
  inflating: data/locations.csv      
  inflating: data/country.csv        
  inflating: data/employees.csv      
  inflating: data/job_history.csv    


Кто получает больше всего? Кто меньше всего?

In [ ]:
df = spark.read.csv("data/employees.csv",sep = "\t",header='True')
df.show(truncate=False)

+-----------+-----------+----------+--------+------------+---------+----------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME |LAST_NAME |EMAIL   |PHONE_NUMBER|HIRE_DATE|JOB_ID    |SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+-----------+----------+--------+------------+---------+----------+------+--------------+----------+-------------+
|100        |Steven     |King      |SKING   |515.123.4567|17.06.03 |AD_PRES   |24000 |null          |null      |90           |
|101        |Neena      |Kochhar   |NKOCHHAR|515.123.4568|21.09.05 |AD_VP     |17000 |null          |100       |90           |
|102        |Lex        |De Haan   |LDEHAAN |515.123.4569|13.01.01 |AD_VP     |17000 |null          |100       |90           |
|103        |Alexander  |Hunold    |AHUNOLD |590.423.4567|03.01.06 |IT_PROG   |9000  |null          |102       |60           |
|104        |Bruce      |Ernst     |BERNST  |590.423.4568|21.05.07 |IT_PROG   |6000  |null          |103       

In [ ]:
new_df = df.select("FIRST_NAME", "LAST_NAME", "SALARY")
changedTypedf = new_df.withColumn("SALARY", new_df["SALARY"].cast("double"))
salary_df = changedTypedf.sort(col("SALARY").desc())

print("MAX SALARY:", salary_df.first()[0],salary_df.first()[1],salary_df.first()[2])
print("MIN SALARY:", salary_df.collect()[salary_df.count()-1][0],salary_df.collect()[salary_df.count()-1][1],salary_df.collect()[salary_df.count()-1][2])

MAX SALARY: Steven King 24000.0
MIN SALARY: TJ Olson 2100.0


Выведете топ 5 по зарплате.

In [ ]:
salary_df.take(5)
for i in salary_df.take(5):
  print(i[0],i[1],i[2])

salary_df.show(5,truncate=False)

Steven King 24000.0
Lex De Haan 17000.0
Neena Kochhar 17000.0
John Russell 14000.0
Karen Partners 13500.0
+----------+---------+-------+
|FIRST_NAME|LAST_NAME|SALARY |
+----------+---------+-------+
|Steven    |King     |24000.0|
|Neena     |Kochhar  |17000.0|
|Lex       |De Haan  |17000.0|
|John      |Russell  |14000.0|
|Karen     |Partners |13500.0|
+----------+---------+-------+
only showing top 5 rows



Сколько всего регионов? Сколько работников в каждом регионе?

In [ ]:
df_dep = spark.read.csv("data/departments.csv",sep = "\t",header='True')
df_new_dep = df.join(df_dep,on = "DEPARTMENT_ID",how = "inner")

df_dep_loc = spark.read.csv("data/locations.csv",sep = "\t",header='True')
df_new_dep_loc = df_new_dep.join(df_dep_loc,on =  "LOCATION_ID" , how = "inner")

df_dep_loc_cnt = spark.read.csv("data/country.csv",sep = "\t",header='True')
df_new_dep_cnt = df_new_dep_loc.join(df_dep_loc_cnt,on =  "COUNTRY_ID" , how = "inner")

df_dep_loc_reg = spark.read.csv("data/regions.csv",sep = "\t",header='True')
df_new_dep_reg = df_new_dep_cnt.join(df_dep_loc_reg,on =  "REGION_ID" , how = "inner")

changedTypedf_new = df_new_dep_reg.withColumn("REGION_ID", df_new_dep_reg["REGION_ID"].cast("double"))
dep_grup = changedTypedf_new.groupBy("REGION_NAME").count()
dep_grup.show()

dep_grup1 = changedTypedf_new.filter(changedTypedf_new['COUNTRY_NAME']=="China").count()
print("Работников из Китая: " , dep_grup1)

+-----------+-----+
|REGION_NAME|count|
+-----------+-----+
|     Europe|   36|
|   Americas|   70|
+-----------+-----+

Работников из Китая:  0


Выведете всех работников из Китая.

In [ ]:
df_his = spark.read.csv("data/job_history.csv",sep = "\t",header='True')
df_1 = df.drop('DEPARTMENT_ID')
df_new_dep_his = df_1.join(df_his,on = "EMPLOYEE_ID",how = "inner")

df_new_dep = df_new_dep_his.join(df_dep,on = "DEPARTMENT_ID",how = "inner")
df_new_dep_loc = df_new_dep.join(df_dep_loc,on =  "LOCATION_ID" , how = "inner")
df_new_dep_cnt = df_new_dep_loc.join(df_dep_loc_cnt,on =  "COUNTRY_ID" , how = "inner")

changedTypedf_new = df_new_dep_cnt.withColumn("REGION_ID", df_new_dep_cnt["REGION_ID"].cast("double"))
dep_grup = changedTypedf_new.filter(changedTypedf_new['COUNTRY_NAME']=="China").count()
print("Работников из Китая: " , dep_grup)

Работников из Китая:  0


Укажите самую высокооплачиваемою должность.

In [ ]:
df_jobs = spark.read.csv("data/jobs.csv",sep = "\t",header='True')

changedTypedf_new = df_jobs.withColumn("MAX_SALARY", df_jobs["MAX_SALARY"].cast("double")).withColumn("MIN_SALARY", df_jobs["MIN_SALARY"].cast("double"))

from pyspark.sql.functions import col
df1=changedTypedf_new.withColumn("avg", (col("MAX_SALARY")+col("MIN_SALARY"))/2)
df1 = df1.sort("avg")
df1.show()
print("ДОЛЖНОСТЬ: ",df1.collect()[df1.count()-1].JOB_TITLE,"\nЗАРПЛАТА: ",df1.collect()[df1.count()-1].avg)


+----------+--------------------+----------+----------+-------+
|    JOB_ID|           JOB_TITLE|MIN_SALARY|MAX_SALARY|    avg|
+----------+--------------------+----------+----------+-------+
|  ST_CLERK|         Stock Clerk|    2008.0|    5000.0| 3504.0|
|  PU_CLERK|    Purchasing Clerk|    2500.0|    5500.0| 4000.0|
|  SH_CLERK|      Shipping Clerk|    2500.0|    5500.0| 4000.0|
|   AD_ASST|Administration As...|    3000.0|    6000.0| 4500.0|
|    MK_REP|Marketing Represe...|    4000.0|    9000.0| 6500.0|
|    HR_REP|Human Resources R...|    4000.0|    9000.0| 6500.0|
|FI_ACCOUNT|          Accountant|    4200.0|    9000.0| 6600.0|
|AC_ACCOUNT|   Public Accountant|    4200.0|    9000.0| 6600.0|
|   IT_PROG|          Programmer|    4000.0|   10000.0| 7000.0|
|    ST_MAN|       Stock Manager|    5500.0|    8500.0| 7000.0|
|    PR_REP|Public Relations ...|    4500.0|   10500.0| 7500.0|
|    SA_REP|Sales Representative|    6000.0|   12008.0| 9004.0|
|    PU_MAN|  Purchasing Manager|    800

Выведете всех работников связанных с ИТ. Выведете их менеджеров. 

In [ ]:
IT = df.filter(df["JOB_ID"] == "IT_PROG")
managers_company = df.selectExpr("FIRST_NAME as NAME_OF_MAN", "LAST_NAME as LAST_NAME_OF_MAN","EMPLOYEE_ID as MANAGER_ID")
IT_employees_with_man = IT.join(managers_company, on = "MANAGER_ID", how = "inner")
IT_employees_with_man.show()

+----------+-----------+----------+---------+--------+------------+---------+-------+------+--------------+-------------+-----------+----------------+
|MANAGER_ID|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE| JOB_ID|SALARY|COMMISSION_PCT|DEPARTMENT_ID|NAME_OF_MAN|LAST_NAME_OF_MAN|
+----------+-----------+----------+---------+--------+------------+---------+-------+------+--------------+-------------+-----------+----------------+
|       102|        103| Alexander|   Hunold| AHUNOLD|590.423.4567| 03.01.06|IT_PROG|  9000|          null|           60|        Lex|         De Haan|
|       103|        104|     Bruce|    Ernst|  BERNST|590.423.4568| 21.05.07|IT_PROG|  6000|          null|           60|  Alexander|          Hunold|
|       103|        105|     David|   Austin| DAUSTIN|590.423.4569| 25.06.05|IT_PROG|  4800|          null|           60|  Alexander|          Hunold|
|       103|        106|     Valli|Pataballa|VPATABAL|590.423.4560| 05.02.06|IT_PROG|  4800|  

Выведете имя и фамилию работника, его текущую и предыдущую должности и сколько полных недель и дней прошло с момент изменения. 



In [ ]:
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType
from pyspark.sql.types import DateType
from pyspark.sql.functions import unix_timestamp, from_unixtime

df_his = spark.read.csv("data/job_history.csv",sep = "\t",header='True')
new_join = df_his.join(df_jobs,on="JOB_ID",how = 'inner')
our_df = new_join.selectExpr("JOB_ID as LAST_JOB_ID", "JOB_TITLE as LAST_JOB_TITLE","EMPLOYEE_ID")
new_join = df.join(our_df,on = "EMPLOYEE_ID",how = "full")
new_join = new_join.drop_duplicates()
new_join = new_join.withColumn("HIRE_DATE",from_unixtime(unix_timestamp(df.HIRE_DATE, 'dd.MM.yy')).alias('HIRE_DATE'))
new_join = new_join.withColumn('daysSince', F.datediff(F.current_date(), new_join.HIRE_DATE))
new_join = new_join.withColumn("daysSince", new_join["daysSince"].cast(IntegerType()))
new_join=new_join.withColumn("weeksSince", (col("daysSince")/7).cast(IntegerType()))
new_join = new_join.join(df_jobs,on="JOB_ID",how="inner")
our_df = new_join.selectExpr("FIRST_NAME", "LAST_NAME","JOB_TITLE","LAST_JOB_TITLE","daysSince","weeksSince")
our_df.show()


+-----------+---------+--------------------+-----------------+---------+----------+
| FIRST_NAME|LAST_NAME|           JOB_TITLE|   LAST_JOB_TITLE|daysSince|weeksSince|
+-----------+---------+--------------------+-----------------+---------+----------+
|       John|  Russell|       Sales Manager|             null|     5927|       846|
|      James|   Landry|         Stock Clerk|             null|     5092|       727|
|      Peter|     Hall|Sales Representative|             null|     5604|       800|
|    Lindsey|    Smith|Sales Representative|             null|     5767|       823|
|    Michael|   Rogers|         Stock Clerk|             null|     5233|       747|
|Jose Manuel|    Urman|          Accountant|             null|     5405|       772|
|      Sarah|     Bell|      Shipping Clerk|             null|     6167|       881|
|    Janette|     King|Sales Representative|             null|     6172|       881|
|     Sundar|     Ande|Sales Representative|             null|     4657|    

Выведете уникальные телефонные номера

In [ ]:
df.select("PHONE_NUMBER").distinct().collect()

[Row(PHONE_NUMBER='011.44.1344.429018'),
 Row(PHONE_NUMBER='515.127.4566'),
 Row(PHONE_NUMBER='515.127.4564'),
 Row(PHONE_NUMBER='011.44.1344.429278'),
 Row(PHONE_NUMBER='515.123.4569'),
 Row(PHONE_NUMBER='650.124.1434'),
 Row(PHONE_NUMBER='650.123.2234'),
 Row(PHONE_NUMBER='011.44.1344.498718'),
 Row(PHONE_NUMBER='650.127.1634'),
 Row(PHONE_NUMBER='515.127.4561'),
 Row(PHONE_NUMBER='011.44.1345.629268'),
 Row(PHONE_NUMBER='515.127.4562'),
 Row(PHONE_NUMBER='011.44.1644.429264'),
 Row(PHONE_NUMBER='011.44.1644.429262'),
 Row(PHONE_NUMBER='650.501.1876'),
 Row(PHONE_NUMBER='650.127.1834'),
 Row(PHONE_NUMBER='011.44.1343.529268'),
 Row(PHONE_NUMBER='011.44.1644.429265'),
 Row(PHONE_NUMBER='515.123.8181'),
 Row(PHONE_NUMBER='650.507.9833'),
 Row(PHONE_NUMBER='515.123.7777'),
 Row(PHONE_NUMBER='650.123.4234'),
 Row(PHONE_NUMBER='515.123.5555'),
 Row(PHONE_NUMBER='011.44.1344.467268'),
 Row(PHONE_NUMBER='650.505.1876'),
 Row(PHONE_NUMBER='650.501.2876'),
 Row(PHONE_NUMBER='515.123.8080'),
 

Есть ли сотрудники с одинаковыми фамилиями и сколько их.

In [ ]:
df.groupBy("LAST_NAME").count().where("count > 1").show()

+---------+-----+
|LAST_NAME|count|
+---------+-----+
|    Smith|    2|
|     King|    2|
|Cambrault|    2|
|   Taylor|    2|
+---------+-----+

